In [ ]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
!pip install git+https://github.com/huggingface/transformers
!pip install ohmeow-blurr
!pip install relevanceai

In [ ]:
token = "" #@param  {type:"string"}

In [ ]:
from relevanceai.utils import decode_workflow_token

config = decode_workflow_token(token)

# config={
#     'dataset_id': "data-csv-1",
#     'text_field': "Response",
#     'output_field': "Rating",
#     'authorizationToken':'',
# }

In [ ]:
"""
NPS Label
"""
import sys
import requests

from collections import defaultdict
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from relevanceai.operations_new.transform_base import TransformBase
from relevanceai.operations_new.processing.text.clean.transform import CleanTextTransform
from relevanceai.constants.stopwords import STOPWORDS
from transformers import *
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

from transformers import *
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [ ]:
class RatingTransform(TransformBase):
    def __init__(
        self,
        text_fields: list,
        output_fields: list = None,
        **kwargs,
    ):
        self.text_fields = text_fields
        self.output_fields = output_fields
        for k, v in kwargs.items():
            setattr(self, k, v)
        



    def rating(
        self,
        text: str,
    ):
        """
        rating the text
        """
        # print(text)
        if text is None:
            return {}
        output = self.predictor.blurr_predict(text)[0]['label']
        # [{'label': 'desire', 'score': 0.30693167448043823}]
        return output 

    def transform(self, documents):
        # For each document, update the field
        response = requests.get(
            "https://userdata.ap-southeast-2.relevance.ai/files/public/379a70fa08457e03a036/nps-model/0bad54a3-55b3-4557-870e-eec5b48279da.pkl"
        )
        with open("model.pkl", 'wb') as f:
            f.write(response.content)

        self.predictor = load_learner(fname="model.pkl")
        docs = [{"_id": d["_id"]} for d in documents]
        ratings = [
                self.rating(
                    self.get_field(self.text_fields[0], doc, missing_treatment="return_empty_string"),
                )
                for doc in documents
            ]
        # print(self.output_fields)
        # print(ratings)
        # print(docs)
        self.set_field_across_documents(self.output_fields[0], ratings, docs)
        return docs

    @property
    def name(self):
        return "Rating"

In [ ]:
from relevanceai.operations_new.ops_base import OperationAPIBase


class RatingOps(OperationAPIBase, RatingTransform):
    """ 
    Add ratings
    """

    def __init__(
        self,
        text_fields: list,
        output_fields: list = None,
        **kwargs,
    ):
        self.text_fields = text_fields
        self.output_fields = output_fields
        super().__init__(**kwargs)

    @property
    def name(self):
        return "rating"

In [ ]:
def extract_rating(
    dataset,
    text_fields: list,
    filters: list = None,
    chunksize: int = 100,
    output_fields: list = None,
    batched: bool = True,
    refresh: bool = False,
):

    ops = RatingOps(
        credentials=dataset.credentials,
        text_fields=text_fields,
        output_fields=output_fields,
    )
    ops.run(
        dataset,
        filters=filters,
        select_fields=text_fields,
        chunksize=chunksize,
        batched=batched,
        output_fields=output_fields,
        refresh=refresh,
    )
    return ops

In [ ]:
dataset_id = config['dataset_id']

text_fields = [config['text_field']]

# model_name = config['model_id']
output_fields = [config.get("output_field")]

from relevanceai import Client
client = Client(token=config['authorizationToken'])

ds = client.Dataset(dataset_id)

In [ ]:
extract_rating(
    dataset=ds,
    text_fields = text_fields,
    output_fields=output_fields,
)